The aim of this notebook is to determine the survival rate of technologies in files after an other technology has been introduced

In [1]:
load.ivy("joda-time" % "joda-time" % "2.8")

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT from sonatype-snapshots, using Fri Jun 05 10:12:58 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Thu Sep 10 11:11:45 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT from sonatype-snapshots, using Mon Jun 01 02:54:01 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT



In [68]:
import java.io.File
import org.joda.time.{DateTime, DateTimeZone, Duration, Interval, Days}
import org.joda.time.format.DateTimeFormat
import scala.io.Source
import java.io.PrintWriter

import java.io.File
import org.joda.time.{DateTime, DateTimeZone, Duration, Interval, Days}
import org.joda.time.format.DateTimeFormat
import scala.io.Source
import java.io.PrintWriter

In [17]:
val directory = new File("/Users/mg/CloudStation/publications/2016/saner2016")
val first_last_file = new File(directory, "data/first-last.summary.csv")
val project_summary_file = new File(directory, "data/projects.summary.csv")
val dtf = DateTimeFormat.forPattern("yyyy-MM-dd HH:mm:ss").withZoneUTC

directory: File = /Users/mg/CloudStation/publications/2016/saner2016
first_last_file: File = /Users/mg/CloudStation/publications/2016/saner2016/data/first-last.summary.csv
project_summary_file: File = /Users/mg/CloudStation/publications/2016/saner2016/data/projects.summary.csv
dtf: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@62fac41b

In [93]:
implicit def orderingByDateTime[A <: DateTime]: Ordering[A] = Ordering.by(d => d.getMillis)

case class Row(project: String, 
               file: String,
               technologies: Map[String, Interval])
{
    def is_entity_related(): Boolean = !technologies.isEmpty
    
   
    
    def is_jpa(t: DateTime): Boolean = technologies.keySet contains "JPA"
    
    def is_jdbc(t: DateTime): Boolean = technologies.keySet contains "JDBC"
    
    def is_hbm(t: DateTime): Boolean = technologies.keySet contains "Hibernate"
}

object Row
{
    def apply(line: String) = {
        val elements = line.split(",", -1)
        val project = elements(0)
        val file = elements(1)
        val jpa = elements(2) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval( dtf.parseDateTime(elements(2)) , dtf.parseDateTime(elements(3))))  
        }
        
        val jdbc = elements(4) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval(dtf.parseDateTime(elements(4)) , dtf.parseDateTime(elements(5))))  
        }
        
        val hbm = elements(6) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval(dtf.parseDateTime(elements(6)) , dtf.parseDateTime(elements(7))))  
        }
        
        val technologies = scala.collection.mutable.Map[String, Interval]()
        if(jpa.isDefined) technologies.put("JPA", jpa.get)
        if(jdbc.isDefined) technologies.put("JDBC", jdbc.get)
        if(hbm.isDefined) technologies.put("Hibernate",hbm.get)
        
        new Row(project,file,technologies.toMap)
    }
}

defined function orderingByDateTime
defined class Row
defined object Row

In [94]:
val entries = Source.fromFile(first_last_file)
                    .getLines.drop(1)
                    .filterNot(_.trim.isEmpty)
                    .map(Row.apply)
                    .toList

entries: List[Row] = List(
  Row(
    "codjo_codjo-sample",
    "sample-server/src/main/java/net/codjo/sample/server/broadcast/ComputedPublicationDate.java",
    Map("JDBC" -> 2012-07-09T14:35:56.000Z/2014-09-17T11:43:58.000Z)
  ),
  Row(
    "codjo_codjo-sample",
    "sample-server/src/main/java/net/codjo/sample/server/broadcast/BookSelector.java",
    Map("JDBC" -> 2012-12-18T18:18:47.000Z/2014-09-17T11:43:58.000Z)
  ),
  Row(
    "codjo_codjo-sample",
    "sample-server/src/test/java/net/codjo/sample/server/broadcast/TransactionManager.java"...

In [95]:
val project_end = Source.fromFile(project_summary_file)
                        .getLines.drop(1)
                        .map(l => l.split(","))
                        .map(l => l(0) -> dtf.withZoneUTC.parseDateTime(l.last.substring(0,19)))
                        .toMap

project_end: Map[String, DateTime] = Map(
  "Netflix_astyanax" -> 2015-01-26T12:52:19.000Z,
  "stewartsims_KMS-dev" -> 2011-08-12T15:22:33.000Z,
  "limcheekin_vaadin-addressbook-sample" -> 2012-06-18T10:49:00.000Z,
  "apache_maven-plugins" -> 2015-03-18T22:11:07.000Z,
  "gphat_bullfinch" -> 2012-06-26T11:15:32.000Z,
  "sagioto_forum" -> 2012-07-30T18:12:20.000Z,
  "jpaoletti_java-presentation-manager" -> 2013-06-25T19:57:22.000Z,
  "qyb_sohu" -> 2012-03-30T16:17:54.000Z,
  "ebayopensource_turmeric-releng" -> 2012-01-12T09:44:28.000Z,
  "restsql_restsql" -> 2015-02-16T14:46:20.000Z,
  "openshift_kitchensink-example" -> 2014-03-31T10:02:07.000Z,
  "hamnis_httpcache4j" -> 2015-02-25T23:01:51.000Z,
  "Tagette_BossStory" -> 2014-12-22T16:07:46.000Z,
  "eclipse_cdo" -> 2015-03-18T08:05:25.000Z,
...

Release the survival data of a sequence of values.
Each value is a duration, plus a boolean expressing if the event actually occured

In [115]:
def release_survival(values: Seq[(Int, Boolean)], file: File) =
{
    val out = new PrintWriter(file)
    out.println("duration,event")
    values.foreach(value => {
        out.println(value._1 + "," + (if(value._2) 1 else 0))
    })
    
    out.flush()
    out.close()
}

defined function release_survival

In [128]:
val technologies = Seq("JPA", "JDBC", "Hibernate")

technologies.foreach(t1 => {
    technologies.foreach(t2 => {
        if(t1 != t2)
        {
            val candidates = entries.filter(p => (p.technologies.keySet contains t1) && 
                                                 (p.technologies.keySet contains t2) && 
                                                !(p.technologies(t1).getStart isAfter p.technologies(t2).getStart) &&
                                                 
                                                 (p.technologies(t1).getEnd isAfter p.technologies(t2).getStart)
                                           )
            
            val values = candidates.map(candidate => {
                val duration = Days.daysBetween(candidate.technologies(t2).getStart,
                                                candidate.technologies(t1).getEnd)
                                   .getDays
                assert(duration >= 0)
                val event = candidate.technologies(t1).getEnd isBefore project_end(candidate.project)
                
                (duration, event)
            })
            
            println(t1 + " -> " + t2 + " : " + candidates.size + " files" )
            release_survival(values, new File(directory, "data/survival-" + 
                                              t1.toLowerCase + "-" + t2.toLowerCase + ".csv"))
        }
    })
})

JPA -> JDBC : 664 files
JPA -> Hibernate : 185 files
JDBC -> JPA : 637 files
JDBC -> Hibernate : 155 files
Hibernate -> JPA : 112 files
Hibernate -> JDBC : 170 files


technologies: Seq[String] = List("JPA", "JDBC", "Hibernate")

In [113]:
entries.size

res77: Int = 93842